In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

## Lectura y Preparación del Dataset

In [2]:
data = pd.read_csv('dataset_final.csv', index_col=0)

In [3]:
from sklearn.preprocessing import LabelEncoder

def data_preparation(df, window=3, neutral=True):
    
    # armar las windows para el armado de las features 
    cols1 = ['pos', 'neg', 'neu'] if neutral==True else ['pos', 'neg']
    for col in cols1:
        df[col+'w'] = df[col].rolling(window=window).mean()
    
    # shifteo un día el retorno. las noticias deben estimar el retorno del día posterior.
    df['exc_ret']  = df['exc_ret'].shift(periods=-1)
    df['exc_ret%'] = df['exc_ret%'].shift(periods=-1) # por si lo necesito 
    
    # elimino las filas que quedan con nan por efecto del rolling window
    df.dropna(axis=0, inplace=True)
    
    # encoding del label que indica si hay retorno positivo (1) o negativo (0)
    le = LabelEncoder()
    df['exc_ret'] = le.fit_transform(df['exc_ret'])
    
    # split en train y test por fecha
    cols2 = [x+'w' for x in cols1]
    
    cut_date = '2018-10-31'
    
    X  = df.loc[df.date <= cut_date, cols2]
    y  = df.loc[df.date <= cut_date, 'exc_ret']
    
    Xt = df.loc[df.date > cut_date, cols2]
    yt = df.loc[df.date > cut_date, 'exc_ret']
    
    return X, Xt, y, yt

## Entrenamiento de Classifiers

In [4]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.svm import SVC

In [5]:
def train_classifier(X, Xt, y, yt):
    
    #model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Logistic Regression',
    #              'Catboost', 'Support Vector']
    model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Catboost']
   
    model_init = [RandomForestClassifier(),
                  XGBClassifier(),
                  lgb.LGBMClassifier(),
                  #LogisticRegression(),
                  CatBoostClassifier(),
                  #SVC(probability=True)
                 ]
    
    # Random Forest 
    params1 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth"   : st.randint(2, 25),     # Maximum tree depth for base learners.
    }
    
    # XGBoost y Lightgbm
    params2 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth": st.randint(2, 25),         # Maximum tree depth for base learners.
        "learning_rate": st.uniform(0.01, 0.5), # Boosting learning rate (xgb’s “eta”)
        "colsample_bytree": st.beta(10, 1),     # Subsample ratio of columns when constructing each tree.
        "subsample": st.beta(10, 1),            # Subsample ratio of the training instance.
        "gamma": st.uniform(0, 10),             # Minimum loss reduction required to make a further partition on a leaf node of the tree.
        'reg_alpha': st.uniform(0.05,10),       # L1 regularization term on weights
        "min_child_weight": st.uniform(1,20),   # Minimum sum of instance weight(hessian) needed in a child.
    }
    
    # Logistic Regression
    params3 = {  
        "penalty"     : ['l1', 'l2'],
        "C"           : st.uniform(0.1, 10.),
    }
    
    # Catboost
    params4 = {}
    
    # SVC
    params5 = {
        "C"           : st.uniform(0.1, 10.),
        #"kernel"      : ['lbf', 'linear']
    }
    
    #model_params = [params1, params2, params2, params3, params4, params5]
    model_params = [params1, params2, params2, params4]
    
    train_scores    = []
    test_scores     = []
    best_estimators = []
    best_parameters = []
    
    #kf    = KFold(n_splits=5, shuffle=True, random_state=RS)
    tsp   = TimeSeriesSplit(n_splits=6)
    
    for name, mod, params in zip(model_name, model_init, model_params):
        
        model = mod

        rgrid = RandomizedSearchCV(estimator=model, param_distributions=params, cv=tsp,
                                   scoring='roc_auc', n_iter=50, verbose=1, n_jobs=-1)
        rgrid.fit(X, y)
        
        best_estimators.append(rgrid.best_estimator_)
        best_parameters.append(rgrid.best_params_)
        train_scores.append(rgrid.best_score_)
        
        ppred = rgrid.predict_proba(Xt)
        score = roc_auc_score(y_score=ppred[:, 1], y_true=yt)
        
        test_scores.append(score)
    
    results = pd.DataFrame(
        {'Model'      : model_name,
         'Train Score': train_scores,
         'Test Score' : test_scores,
         'Params'     : best_parameters,
         'Estimator'  : best_estimators
        })
    
    return results

In [6]:
import itertools
windows = np.arange(1, 6)
neutral = [1, 0]

resultado = pd.DataFrame()

for win, neu in itertools.product(windows, neutral):
    
    X, X_test, y, y_test = data_preparation(data, window=win, neutral=neu)
    
    clfs = train_classifier(X, X_test, y, y_test)
  
    res = pd.DataFrame(
    {'Model'      : clfs['Model'],
     'Window'     : list(str(win)*clfs.shape[0]),
     'Neutral'    : list(str(neu)*clfs.shape[0]),
     'Train Score': clfs['Train Score'],
     'Test Score' : clfs['Test Score'],
     'Params'     : clfs['Params'],
     #'Estimator'  : clfs['Estimator']
    })
    
    resultado = pd.concat([resultado, res], axis=0)

Fitting 6 folds for each of 50 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    8.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 6 folds for each of 50 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 6 folds for each of 50 candidates, totalling 300 fits


...

343:	learn: 0.5609482	total: 2.96s	remaining: 5.64s
344:	learn: 0.5607266	total: 2.97s	remaining: 5.63s
345:	learn: 0.5603155	total: 2.97s	remaining: 5.62s
346:	learn: 0.5598044	total: 2.98s	remaining: 5.61s
347:	learn: 0.5594744	total: 2.99s	remaining: 5.59s
348:	learn: 0.5594612	total: 2.99s	remaining: 5.58s
349:	learn: 0.5592007	total: 3s	remaining: 5.57s
350:	learn: 0.5589218	total: 3.01s	remaining: 5.56s
351:	learn: 0.5587080	total: 3.01s	remaining: 5.55s
352:	learn: 0.5584066	total: 3.02s	remaining: 5.54s
353:	learn: 0.5582201	total: 3.03s	remaining: 5.53s
354:	learn: 0.5579649	total: 3.04s	remaining: 5.51s
355:	learn: 0.5578711	total: 3.04s	remaining: 5.5s
356:	learn: 0.5575830	total: 3.05s	remaining: 5.49s
357:	learn: 0.5575153	total: 3.06s	remaining: 5.48s
358:	learn: 0.5572730	total: 3.06s	remaining: 5.47s
359:	learn: 0.5568726	total: 3.07s	remaining: 5.46s
360:	learn: 0.5568323	total: 3.08s	remaining: 5.45s
361:	learn: 0.5565717	total: 3.08s	remaining: 5.43s
362:	learn: 0.55

506:	learn: 0.5302382	total: 4.13s	remaining: 4.02s
507:	learn: 0.5301997	total: 4.14s	remaining: 4.01s
508:	learn: 0.5300987	total: 4.15s	remaining: 4s
509:	learn: 0.5299742	total: 4.16s	remaining: 3.99s
510:	learn: 0.5296923	total: 4.16s	remaining: 3.98s
511:	learn: 0.5295400	total: 4.17s	remaining: 3.97s
512:	learn: 0.5294152	total: 4.18s	remaining: 3.97s
513:	learn: 0.5291041	total: 4.18s	remaining: 3.96s
514:	learn: 0.5288676	total: 4.19s	remaining: 3.95s
515:	learn: 0.5288652	total: 4.2s	remaining: 3.94s
516:	learn: 0.5286968	total: 4.2s	remaining: 3.93s
517:	learn: 0.5285958	total: 4.21s	remaining: 3.92s
518:	learn: 0.5285396	total: 4.22s	remaining: 3.91s
519:	learn: 0.5283749	total: 4.22s	remaining: 3.9s
520:	learn: 0.5283715	total: 4.23s	remaining: 3.89s
521:	learn: 0.5281415	total: 4.24s	remaining: 3.88s
522:	learn: 0.5279010	total: 4.24s	remaining: 3.87s
523:	learn: 0.5276404	total: 4.25s	remaining: 3.86s
524:	learn: 0.5275847	total: 4.26s	remaining: 3.85s
525:	learn: 0.5273

674:	learn: 0.5047435	total: 5.31s	remaining: 2.56s
675:	learn: 0.5047231	total: 5.31s	remaining: 2.55s
676:	learn: 0.5046708	total: 5.32s	remaining: 2.54s
677:	learn: 0.5046322	total: 5.33s	remaining: 2.53s
678:	learn: 0.5045525	total: 5.33s	remaining: 2.52s
679:	learn: 0.5044028	total: 5.34s	remaining: 2.51s
680:	learn: 0.5039651	total: 5.35s	remaining: 2.51s
681:	learn: 0.5035813	total: 5.36s	remaining: 2.5s
682:	learn: 0.5035048	total: 5.36s	remaining: 2.49s
683:	learn: 0.5030305	total: 5.37s	remaining: 2.48s
684:	learn: 0.5028571	total: 5.38s	remaining: 2.47s
685:	learn: 0.5028244	total: 5.38s	remaining: 2.46s
686:	learn: 0.5025033	total: 5.39s	remaining: 2.46s
687:	learn: 0.5024787	total: 5.39s	remaining: 2.45s
688:	learn: 0.5023359	total: 5.4s	remaining: 2.44s
689:	learn: 0.5021999	total: 5.41s	remaining: 2.43s
690:	learn: 0.5021701	total: 5.42s	remaining: 2.42s
691:	learn: 0.5018244	total: 5.42s	remaining: 2.41s
692:	learn: 0.5017914	total: 5.43s	remaining: 2.4s
693:	learn: 0.5

835:	learn: 0.4823097	total: 6.47s	remaining: 1.27s
836:	learn: 0.4822066	total: 6.48s	remaining: 1.26s
837:	learn: 0.4821201	total: 6.49s	remaining: 1.25s
838:	learn: 0.4820691	total: 6.49s	remaining: 1.25s
839:	learn: 0.4819821	total: 6.5s	remaining: 1.24s
840:	learn: 0.4815918	total: 6.51s	remaining: 1.23s
841:	learn: 0.4814826	total: 6.52s	remaining: 1.22s
842:	learn: 0.4812429	total: 6.52s	remaining: 1.21s
843:	learn: 0.4812029	total: 6.53s	remaining: 1.21s
844:	learn: 0.4811861	total: 6.54s	remaining: 1.2s
845:	learn: 0.4809028	total: 6.54s	remaining: 1.19s
846:	learn: 0.4808315	total: 6.55s	remaining: 1.18s
847:	learn: 0.4807910	total: 6.56s	remaining: 1.18s
848:	learn: 0.4805643	total: 6.56s	remaining: 1.17s
849:	learn: 0.4805093	total: 6.57s	remaining: 1.16s
850:	learn: 0.4804552	total: 6.58s	remaining: 1.15s
851:	learn: 0.4803054	total: 6.58s	remaining: 1.14s
852:	learn: 0.4801875	total: 6.59s	remaining: 1.14s
853:	learn: 0.4799887	total: 6.6s	remaining: 1.13s
854:	learn: 0.4

998:	learn: 0.4638028	total: 7.63s	remaining: 7.64ms
999:	learn: 0.4637842	total: 7.64s	remaining: 0us


In [7]:
res = resultado.sort_values(by='Train Score', ascending=False)
res

,Model,Window,Neutral,Train Score,Test Score,Params
0,Random Forest,3,1,0.586641,0.791667,"{'max_depth': 22, 'n_estimators': 50}"
0,Random Forest,2,1,0.581503,0.505495,"{'max_depth': 8, 'n_estimators': 52}"
2,LightGBM,1,0,0.572187,0.572115,"{'colsample_bytree': 0.8567967099486709, 'gamma': 8.534656778088218, 'learning_rate': 0.052298456335397196, 'max_depth': 22, 'min_child_weight': 1.7985308860322138, 'n_estimators': 13, 'reg_alpha': 0.5584806163085043, 'subsample': 0.9282360898528735}"
3,Catboost,2,1,0.571687,0.505495,{}
0,Random Forest,5,0,0.571073,0.450000,"{'max_depth': 10, 'n_estimators': 13}"
0,Random Forest,5,1,0.570609,0.511111,"{'max_depth': 16, 'n_estimators': 22}"
2,LightGBM,4,0,0.564141,0.440000,"{'colsample_bytree': 0.9580338418731638, 'gamma': 7.484319325421484, 'learning_rate': 0.34932882945259697, 'max_depth': 19, 'min_child_weight': 6.700056305022645, 'n_estimators': 72, 'reg_alpha': 1.1195119520567498, 'subsample': 0.993489693165233}"
0,Random Forest,4,0,0.558485,0.480000,"{'max_depth': 2, 'n_estimators': 39}"
1,XGBoost,5,1,0.554271,0.733333,"{'colsample_bytree': 0.9677886140318158, 'gamma': 0.29812378019125507, 'learning_rate': 0.11896181984866581, 'max_depth': 5, 'min_child_weight': 2.9287197326904626, 'n_estimators': 11, 'reg_alpha': 2.294709137524845, 'subsample': 0.9433852978052838}"
1,XGBoost,1,0,0.553624,0.548077,"{'colsample_bytree': 0.95568875502126, 'gamma': 0.0750450877183384, 'learning_rate': 0.03435430359388835, 'max_depth': 3, 'min_child_weight': 3.1497172488855942, 'n_estimators': 43, 'reg_alpha': 2.899139079542998, 'subsample': 0.9809416558401751}"


In [8]:
res.to_csv('resultados.csv')